In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import os
os.chdir(r"/Users/naghamdaood/Projects/Adapting-Recommender-Fashion")

import pandas as pd
from resources.constants import *

outfits_df = pd.read_csv(OUTFITS_CSV_PATH, sep=';')
outfits_df.head()

,id,name,description,group,owner,timeCreated,retailPrice,outfit_tags,tag_categories
0,outfit.fffdaa715c3646f8b1c0f04d549ff07e,Out of stock - Asymmetric Frilled Dress,"This fun, short dress features and asymmetric ...",group.50a586c78eb7626e294ba3bd07d12c79,o_00464,2017-12-30 11:28:01.000,4000.0,"['Synthetic', 'Statement', 'Dresses', 'Metalli...","['Material', 'Occasion', 'Category', 'Details'..."
1,outfit.fffa1b9a3db6415d806f3c48f8ab58d9,Yellow Shell Mellomholmene Blouse,This beautiful blouse features an adjustable n...,group.61ad2fcabb3e9197e3836376e6b67f2c,o_00407,2021-06-07 12:07:22.921,1300.0,"['ILAG', 'Tops', 'Spring', 'Summer', 'M', 'Pat...","['Brand', 'Category', 'Seasons', 'Seasons', 'S..."
2,outfit.fff175b13ceb453f9928625491412ede,Kaula Dress Black,Kaula from Rodebjer is a fitted dress made in ...,group.37c2b59d63d3a9c2d58e07f532f71f7f,o_00359,2023-06-05 09:17:59.004,3100.0,"['Black', 'Mini', 'M', 'Everyday', 'Multi Seas...","['Color', 'Length', 'Size', 'Occasion', 'Seaso..."
3,outfit.ffef9d7c292a48b69076d2df2e32352f,For sale - Jarvis Blouse,This wrap blouse has mid length sleeves and a ...,group.dfcaa57546b0b7a5e9eb204449b6cc1c,o_00661,2021-05-18 14:02:28.690,1500.0,"['XS', 'Multi Season', 'Stylein', 'Tops', 'Cot...","['Size', 'Seasons', 'Brand', 'Category', 'Mate..."
4,outfit.ffeef842238f4dbdabc6c730a75aa2bd,Black Amber Pants,"Feel slack and nice dressed with this pant, ma...",group.ee297c977905eb21a123a4aea5fbb6d2,o_00158,2021-07-16 14:02:30.643,1200.0,"['Cotton', 'Black', 'Everyday', 'Knitwear', 'L...","['Material', 'Color', 'Occasion', 'Category', ..."


In [2]:
third_chance_df = pd.read_csv(THIRD_CHANCE_CSV_PATH, sep=';')
print(third_chance_df.shape)
third_chance_df.head()

(1244, 10)


,date_added,id,name,brand,owner,condition,condition_desctription,retail_price,tc_price,sold
0,NaN,outfit.00731d8db5504c64af534f9f43a7f061,L Heaston Skirt Hushed Violet,Samsøe Samsøe,FJONG,5 - Damaged,Flekker som ikke går bort.,800,50.0,True
1,2022-06-10,outfit.00fa8d1fd5804fe78ac284c2e0c28b4e,L Navy Buffalo Skirt,Holzweiler,FJONG,5 - Damaged,Small black dots on shoulders and back (see im...,4700,1175.0,False
2,2023-10-31,outfit.0106cbd7e23f41da92e3081146b4ad03,Juliette Skirt Red Line,Samsøe & Samsøe,FJONG,5 - Damaged,Store flekker i front,2900,580.0,False
3,2022-08-18,outfit.0113e4422a3f408f8fc9a94d2861474f,Sand Poddle99 Blazer,Holzweiler,Holzweiler,5 - Damaged,Lite Hull både foran og bak. Kan sys? eventuel...,2400,300.0,False
4,2023-01-10,outfit.0118b605888840348f80965c2d8865cf,XS Wabi Embroidered Bright White Shirt,FWSS,FWSS,5 - Damaged,NaN,NaN,NaN,False


In [3]:
third_chance_df["date_added"] = pd.to_datetime(third_chance_df["date_added"], errors='coerce', yearfirst=True)

third_chance_df = third_chance_df.dropna(subset=["date_added", "id"])
third_chance_df = third_chance_df.drop_duplicates(subset=["id"])
third_chance_df = third_chance_df.merge(outfits_df, left_on="id", right_on="id", how="inner", suffixes=(None, "_outfits"))#.drop_duplicates(subset=["id_outfits"])
third_chance_df["timeCreated"] = pd.to_datetime(third_chance_df["timeCreated"])
third_chance_df["time_diff"] = third_chance_df["date_added"] - third_chance_df["timeCreated"]
third_chance_df["time_diff"].describe()

count                            983
mean     884 days 02:07:25.600004064
std      613 days 16:20:55.556343352
min        -34 days +15:26:16.051000
25%         412 days 11:20:04.712000
50%         664 days 10:56:54.345000
75%        1213 days 00:22:02.500000
max               2601 days 10:49:59
Name: time_diff, dtype: object

In [4]:
import re
NUMERIC_PATTERN = re.compile(r"(\d+)")

def find_numeric_condition(x):
    if type(x) != str:
        return None

    match = NUMERIC_PATTERN.search(x)
    if match:
        return int(match.group(1))
    return None

third_chance_df["condition_num"] = third_chance_df["condition"].apply(find_numeric_condition)

In [5]:
third_chance_outfits_df = outfits_df[outfits_df["id"].isin(third_chance_df["id"])]
third_chance_outfits_df = third_chance_outfits_df.merge(third_chance_df, left_on="id", right_on="id", how="inner", validate="one_to_one", suffixes=(None, "_third_chance"))
third_chance_outfits_df.drop(third_chance_outfits_df.filter(regex='_third_chance$').columns.tolist(), axis=1, inplace=True)

In [6]:
third_chance_outfits_df[third_chance_outfits_df["condition_num"] >= 3]

,id,name,description,group,owner,timeCreated,retailPrice,outfit_tags,tag_categories,date_added,brand,condition,condition_desctription,retail_price,tc_price,sold,name_outfits,owner_outfits,time_diff,condition_num
72,outfit.e9754e1dba324755a1e327c3105fd036,"The Long Sleeved Kaftan Dress, Black",Say hello to the Long Sleeved Kaftan - our mos...,group.de3bd07fdee460d07dbe02df9a5043d6,o_00096,2021-09-14 13:32:40.966,1800.0,"['Everyday', 'Fall', 'Statement', 'Black', 'Su...","['Occasion', 'Seasons', 'Occasion', 'Color', '...",2023-12-12,AWAN,4 - Minor defects,Stains,4700,1500.0,False,"The Long Sleeved Kaftan Dress, Black",o_00096,818 days 10:27:19.034000,4.0
73,outfit.e965f8f929a345ecab825313cd4b3f24,S1 Gardenia Elysian Shirt,An elegant and feminine blouse with ruffle det...,group.4df8a3e68d3bf1b1fb825e729f2acd44,o_00302,2020-07-03 10:08:04.005,1200.0,"['Ruffles', 'Everyday', 'Tops', 'Women', 'Shir...","['Details', 'Occasion', 'Category', 'Gender', ...",2023-12-12,Karen by Simonsen,4 - Minor defects,Wear and tear,1400,400.0,False,S1 Gardenia Elysian Shirt,o_00302,1256 days 13:51:55.995000,4.0
74,outfit.e94be40cf46643c59bc777a9cb3f58fa,1 Cameron Dotted Blouse,"This delicate dotted blouse comes in a soft, s...",group.0b94d4e8c8c93842903d175da87331c2,o_00661,2020-10-15 13:05:23.184,2000.0,"['Ruffles', 'Multi Season', 'Everyday', 'S', '...","['Details', 'Seasons', 'Occasion', 'Size', 'Co...",2024-01-25,Ganni,4 - Minor defects,Wear and tear,600,250.0,False,1 Cameron Dotted Blouse,o_00661,1196 days 10:54:36.816000,4.0
75,outfit.e9319dccf9234668a6951834dcaff863,Isabella Wrap Top Mint Dandy,"The Isabella wrap top features a peplum waist,...",group.2780c47a10e8be907908012828c7afd5,o_00237,2019-02-05 13:43:11.000,2700.0,"['Everyday', 'Silk', 'Wrap', 'FWSS', 'Tops', '...","['Occasion', 'Material', 'Fit', 'Brand', 'Cate...",2022-09-26,FWSS,4 - Minor defects,Wear and tear,1700,500.0,False,Isabella Wrap Top Mint Dandy,o_00237,1328 days 10:16:49,4.0
76,outfit.e8bc2a3b91ad4a7c8745b2eb4031dd09,Marian Swirl Top Raw Umbra,Marian Swirl is a polo neck in a luxurious vis...,group.19c32f71494778d391423fdf86b661c2,o_00753,2022-07-19 09:55:06.673,1800.0,"['Rodebjer', 'Brown', 'Winter', 'Fall', 'Red',...","['Brand', 'Color', 'Seasons', 'Seasons', 'Colo...",2024-01-25,Rodebjer,3 - Good condition,NaN,4650,1400.0,False,Marian Swirl Top Raw Umbra,o_00753,554 days 14:04:53.327000,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,outfit.01738c87529546bfb8c032dc16027359,Shoulder Embellished Tunic,The Platinum Shoulder Embellished Tunic in Pla...,group.e0f2a30dd908deca2960f51ba031890e,o_00556,2022-05-23 14:30:01.014,2700.0,"['Everyday', 'Dressed-up', 'Grey', 'Women', 'S...","['Occasion', 'Occasion', 'Color', 'Gender', 'S...",2023-05-15,SoyaAulait,5 - Damaged,NaN,1700,340.0,False,Shoulder Embellished Tunic,o_00556,356 days 09:29:58.986000,5.0
979,outfit.0118b605888840348f80965c2d8865cf,Wabi Embroidered Bright White Shirt,The FWSS Wabi Embroidery Shirt is a crisp 100%...,group.92a5ab7fdf4cc111e8f4ae277d9cd013,o_00160,2022-06-22 04:12:07.947,2000.0,"['Spring', 'Cotton', 'Business', 'Dressed-up',...","['Seasons', 'Material', 'Occasion', 'Occasion'...",2023-01-10,FWSS,5 - Damaged,NaN,NaN,NaN,False,Wabi Embroidered Bright White Shirt,o_00160,201 days 19:47:52.053000,5.0
980,outfit.0113e4422a3f408f8fc9a94d2861474f,Sand Poddle99 Blazer,This regular fitted blazer features two button...,group.1a44560b56290e6039f49dc083645a0e,o_00120,2021-08-27 12:30:04.490,3500.0,"['Beige', 'Holzweiler x Vibrent', 'Multi Seaso...","['Color', 'Brand', 'Seasons', 'Material', 'Occ...",2022-08-18,Holzweiler,5 - Damaged,Lite Hull både foran og bak. Kan sys? eventuel...,2400,300.0,False,Sand Poddle99 Blazer,o_00120,355 days 11:29:55.510000,5.0
981,outfit.0106cbd7e23f41da92e3081146b4ad03,Juliette Skirt Red Line,"The Juliette skirt is the perfect, flowy, ligh...",group.6c4aa786f2b635e75d1c9853dfaa47fe,o_00237,2019-02-19 10:00:52.000,1300.0,"['Stripes', 'Skirt

In [7]:
triplets_df = pd.read_csv(USER_ACTIVITY_TRIPLETS_CSV_PATH, sep=CSV_SEPARATOR)
triplets_df.head()

,customer.id,outfit.id,rentalPeriod.start,rentalPeriod.end
0,3741,outfit.5c081909537b42239e465d2d615c705f,2023-03-26,2023-04-25
1,6563,outfit.c34969dd8b334064aa90bfb60c8ec308,2023-03-27,2023-04-26
2,6563,outfit.aef4cc93eebf40ca8820790deb7a8323,2023-01-27,2023-02-26
3,5125,outfit.3de5df48a14b4a9aba6d8e41d11e9351,2021-11-16,2021-12-15
4,5125,outfit.0eaa358af14e469894062591bd42f38b,2021-10-11,2021-11-11


In [8]:
triplets_df = pd.read_csv(USER_ACTIVITY_TRIPLETS_CSV_PATH, sep=CSV_SEPARATOR)
rentals_count_dict = triplets_df["outfit.id"].value_counts().to_dict()
def get_rentals_count(x):
    return rentals_count_dict.get(x, 0)

third_chance_outfits_df["rentals_count"] = third_chance_outfits_df["id"].map(get_rentals_count)

In [9]:
third_chance_outfits_df[["time_diff", "condition_num", "rentals_count"]].corr()

,time_diff,condition_num,rentals_count
time_diff,1.000000,-0.193208,0.470524
condition_num,-0.193208,1.000000,-0.020954
rentals_count,0.470524,-0.020954,1.000000


In [10]:
import ast

def convert_string_to_list(x):
    if type(x) != str:
        return []
    return ast.literal_eval(x)

third_chance_outfits_df["outfit_tags"] = third_chance_outfits_df["outfit_tags"].apply(convert_string_to_list)
third_chance_outfits_df["tag_categories"] = third_chance_outfits_df["tag_categories"].apply(convert_string_to_list)

In [11]:
import numpy as np

def get_outfit_category(tag_categories, tags, category):
    tag_categories, tags = np.array(tag_categories), np.array(tags)
    category_indexes = np.where(tag_categories == category)[0]
    if len(category_indexes) == 0:
        return []
    cat_tags = tags[category_indexes]
    return cat_tags
    
    # try:
    #     cat_index = tag_categories.index(category)
    #     print(cat_index, len(tags), tag_categories, tags)
    #     return tags[cat_index]
    # except ValueError:
    #     return None

third_chance_outfits_df["brand"] = third_chance_outfits_df.apply(lambda x: get_outfit_category(x["tag_categories"], x["outfit_tags"], "Brand"), axis=1)
third_chance_outfits_df["material"] = third_chance_outfits_df.apply(lambda x: get_outfit_category(x["tag_categories"], x["outfit_tags"], "Material"), axis=1)
third_chance_outfits_df["size"] = third_chance_outfits_df.apply(lambda x: get_outfit_category(x["tag_categories"], x["outfit_tags"], "Size"), axis=1)

In [12]:
third_chance_outfits_df.sort_values(by="group")

,id,name,description,group,owner,timeCreated,retailPrice,outfit_tags,tag_categories,date_added,...,retail_price,tc_price,sold,name_outfits,owner_outfits,time_diff,condition_num,rentals_count,material,size
421,outfit.93dcb93c8c104b50bcfe3d83b36e1d53,Prenestina Jacket Baby Blue Jacket,"Super elegant, constructed jacket with puffed ...",group.000aad5ac8cfefd8063ee16f6f57b2c1,o_00473,2022-03-28 13:54:38.895,2500.0,"[Viscose, Spring, Everyday, Cotton, Women, Fal...","[Material, Seasons, Occasion, Material, Gender...",2023-05-15,...,2200,849.0,False,Prenestina Jacket Baby Blue Jacket,o_00473,412 days 10:05:21.105000,2.0,0,"[Viscose, Cotton]",[L]
58,outfit.edbe38fccd174c3cac918d1347b7811e,Prenestina Jacket Baby Blue Jacket,"Super elegant, constructed jacket with puffed ...",group.000aad5ac8cfefd8063ee16f6f57b2c1,o_00473,2022-03-28 13:54:38.895,2500.0,"[Women, Viscose, Everyday, Jackets, S, Blue, C...","[Gender, Material, Occasion, Category, Size, C...",2023-01-10,...,2400,650.0,False,Prenestina Jacket Baby Blue Jacket,o_00473,287 days 10:05:21.105000,2.0,3,"[Viscose, Cotton]",[S]
464,outfit.8b5234b3619cf5fb,Hilde Manor Top,This long-sleeved top with luxurious lace embr...,group.00ea09a37a420dbd3b02b973f5fc43c4,o_00237,2017-08-22 16:10:53.000,1800.0,"[Blue, S, FWSS, Synthetic, Women, Everyday, Mu...","[Color, Size, Brand, Material, Gender, Occasio...",2024-01-25,...,1800,549.0,True,Hilde Manor Top,o_00237,2346 days 07:49:07,2.0,6,[Synthetic],[S]
808,outfit.334b32157287403aab73d9cc978a93dd,Bright White Torunn Top,"The Torunn top features embroidered hems, a co...",group.010dfb6d1c3b9a18cbcceb1aa3547b84,o_00146,2020-09-25 10:12:54.755,2500.0,"[White, Blouses, Women, Cotton, Everyday, Tops...","[Color, Category, Gender, Material, Occasion, ...",2022-09-15,...,5300,800.0,True,Bright White Torunn Top,o_00146,719 days 13:47:05.245000,5.0,9,[Cotton],[XS]
438,outfit.90acfab765674a3ea8382b104aba70af,Feather Embroidery Front Pleated Dress,The Feather Embroidery Front Pleated Dress in ...,group.010f6372ca76731e871e244041faa3e2,o_00556,2022-05-23 14:30:01.014,1900.0,"[Dresses, Silk, Summer, Orange, Soyaaulait, Sp...","[Category, Material, Seasons, Color, Brand, Se...",2023-09-19,...,1300,599.0,False,Feather Embroidery Front Pleated Dress,o_00556,483 days 09:29:58.986000,2.0,0,"[Silk, Cotton]",[S]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,outfit.c9b02cffa22042118df20e0d722f6fd5,Sierra Dress Riviera blue,Sierra from ROTATE is a long-sleeved jersey dr...,group.fe5c56eb2d396a69f72bf761729c3fa2,o_00392,2023-04-20 12:00:48.382,2200.0,"[ROTATE Birger Christensen, Stretchy, Dresses,...","[Brand, Fit, Category, Gender, Seasons, Size, ...",2023-12-12,...,NaN,3000.0,True,Sierra Dress Riviera blue,o_00392,235 days 11:59:11.618000,NaN,3,[Synthetic],[S]
524,outfit.7cff339e7950448c8485a2b44205adc0,Bellix Night Blue Blouse,Look simple and luxurious in this silk top. Fe...,group.fe65c72d7049e270ca0242552ecb41e3,o_00237,2018-11-30 12:28:06.000,3000.0,"[S, Silk, Blue, Tops, Everyday, Multi Season, ...","[Size, Material, Color, Category, Occasion, Se...",2022-09-26,...,3000,1200.0,True,Bellix Night Blue Blouse,o_00237,1395 days 11:31:54,2.0,8,[Silk],[S]
365,outfit.9e8dd974e09f44ef886fce0d3557d652,Bellix Night Blue Blouse,Look simple and luxurious in this silk top. Fe...,group.fe65c72d7049e270ca0242552ecb41e3,o_00237,2018-11-30 12:23:43.000,3000.0,"[Women, By Malene Birger, Everyday, Blue, Spri...","[Gender, Brand, Occasion, Color, Seasons, Cate...",2022-09-15,...,900,200.0,False,Bellix Night Blue Blouse,o_00237,1384 days 11:36:17,2.0,6,[Silk],[XL]
101,outfit.e350fdafca034bd3b311d288a391ad41,Isadora Top Lime Green Floral,The Isadora top by Faithful The Brand in Luda ...,group.fe69bb88097e3a80570800493b044ec4,o_00753,2022-06-28 14:57:23.167,1300.0,"[Floral, Women, Ruffles, Faithfull The Brand, ...","[Details, Gender, Details, Brand, Color, Occas...",2024-01-25,...,4500,1400.0,False,Isadora Top Lime Green Floral,o_00753

In [13]:
all_brand_classes = third_chance_outfits_df["brand"].explode()
mlb_brand_classes = all_brand_classes.value_counts()[all_brand_classes.value_counts() > 10].index.to_list()

all_material_classes = third_chance_outfits_df["material"].explode()
mlb_material_classes = all_material_classes.value_counts()[all_material_classes.value_counts() > 10].index.to_list()

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

brand_mlb = MultiLabelBinarizer(classes=mlb_brand_classes)
brand_df = pd.DataFrame(brand_mlb.fit_transform(third_chance_outfits_df["brand"]), columns=["brand_" + mlb_class for mlb_class in brand_mlb.classes_],  index=third_chance_outfits_df.index)

material_mlb = MultiLabelBinarizer()
material_df = pd.DataFrame(material_mlb.fit_transform(third_chance_outfits_df["material"]), columns=["material_" + mlb_class for mlb_class in material_mlb.classes_],  index=third_chance_outfits_df.index)

/opt/anaconda3/envs/fashionEnv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['2NDDAY', 'Acne Studios', 'Adrianna Papell', 'Alaïa', 'Alexander McQueen', 'Altuzarra', 'Ami Amalia', 'Ane Mone', 'Aris', 'Armani', 'Asos', 'Balenciaga', 'Balmain', 'Barbour', 'Bastet Noir', 'Baum und Pferdgarten', 'Billie & me', 'Britt Sisseck', 'Burberry', 'By Malina', 'Cala Jade', 'Camilla Pihl', 'Cardsome', 'Carin Wester', 'Carven', 'Cathrine Hammel', 'Celine', 'Chanel', 'Chloé', 'Christina Ledang', 'Color Vision', 'ConfettiBird', 'Creative Collective', 'Dale of Norway', 'Day Birger et Mikkelsen', 'Designers Remix', 'Diana Orving', 'Diane von Furstenberg', 'Dolce & Gabbana', 'Envelope1976', 'Filippa K', 'Five Units', 'Fleischer Couture', 'Forte Forte', 'Givenchy', 'Gucci', 'H&M', 'H20Fagerholt', 'Hekne', 'Hofmann Copenhagen', 'Houdini', 'Hunkydory', 'IBEN', 'ILAG', 'IRO', 'Ida Sjöstedt', 'InWear', 'Jorun x Vibrent', 'Karen By Simonsen', 'Kari Traa', 'Kenz

In [15]:
third_chance_outfits_df = pd.concat([third_chance_outfits_df, brand_df, material_df], axis=1)
#third_chance_outfits_df = pd.concat([third_chance_outfits_df, material_df], axis=1)

In [16]:
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


from sklearn.preprocessing import MinMaxScaler

brand_columns = [col for col in third_chance_outfits_df.columns if col.startswith("brand_")]
material_columns = [col for col in third_chance_outfits_df.columns if col.startswith("material_")]
LR_COLUMNS = ["condition_num", "rentals_count", "retailPrice"] + brand_columns + material_columns # "tc_price"
PREDICTION_COLUMN = ["time_diff"]
SEED = 44

train_df, test_df = train_test_split(third_chance_outfits_df, test_size=0.2, random_state=SEED)
train_df = train_df[LR_COLUMNS + PREDICTION_COLUMN].copy().fillna(0)
test_df = test_df[LR_COLUMNS + PREDICTION_COLUMN].copy().fillna(0)
train_df["time_diff"] = train_df["time_diff"].dt.days
test_df["time_diff"] = test_df["time_diff"].dt.days

min_max_scaler = MinMaxScaler()
train_df[LR_COLUMNS] = min_max_scaler.fit_transform(train_df[LR_COLUMNS])
test_df[LR_COLUMNS] = min_max_scaler.transform(test_df[LR_COLUMNS])

train_x, train_y = train_df[LR_COLUMNS].values, train_df[PREDICTION_COLUMN].values.ravel()

model = LinearRegression()
#model = LogisticRegression(max_iter=10**3)
#model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=SEED)
model.fit(train_x, train_y)

LinearRegression()

In [17]:
test_x = test_df[LR_COLUMNS].values

test_df["predicted_time_diff"] = model.predict(test_x)
test_df["difference_from_predicted"] = abs(test_df["time_diff"] - test_df["predicted_time_diff"])
test_df["difference_from_predicted"].describe()

count     197.000000
mean      336.862832
std       321.843948
min         0.452245
25%        91.337582
50%       213.128057
75%       472.590971
max      1499.487992
Name: difference_from_predicted, dtype: float64

In [18]:
test_df["difference_from_predicted"].describe()

count     197.000000
mean      336.862832
std       321.843948
min         0.452245
25%        91.337582
50%       213.128057
75%       472.590971
max      1499.487992
Name: difference_from_predicted, dtype: float64

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

class FCN(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size_2, output_size):
        super(FCN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size_2)
        self.layer3 = nn.Linear(hidden_size_2, output_size)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

input_size = train_x.shape[1]
y_scaler = MinMaxScaler()
train_y_normalized = y_scaler.fit_transform(train_y.reshape(-1, 1))

train_x_torch = torch.tensor(train_x, dtype=torch.float32)
train_y_torch = torch.tensor(train_y_normalized, dtype=torch.float32).squeeze()

train_data = TensorDataset(train_x_torch, train_y_torch)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)  # Adjust batch size as needed

input_size = train_x.shape[1]
HIDDEN_SIZE_1 = 100
HIDDEN_SIZE_2 = 100
OUTPUT_SIZE = 1

model = FCN(input_size, HIDDEN_SIZE_1, HIDDEN_SIZE_2, OUTPUT_SIZE)
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression problems
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Learning rate


In [20]:
EPOCHS = 10**2  # Number of epochs

for epoch in range(EPOCHS):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item()}')


Epoch 1/100, Loss: 0.04554755240678787
Epoch 11/100, Loss: 0.027459008619189262
Epoch 21/100, Loss: 0.028544047847390175
Epoch 31/100, Loss: 0.005620359908789396
Epoch 41/100, Loss: 0.007516198791563511
Epoch 51/100, Loss: 0.007003020029515028
Epoch 61/100, Loss: 0.036068178713321686
Epoch 71/100, Loss: 0.022135134786367416
Epoch 81/100, Loss: 0.018305836245417595
Epoch 91/100, Loss: 0.013900581747293472


In [21]:
# Predictions
test_x_torch = torch.tensor(test_x, dtype=torch.float32)
with torch.no_grad():
    model.eval()
    test_predictions = model(test_x_torch).numpy().ravel()

test_df["predicted_time_diff"] = y_scaler.inverse_transform(test_predictions.reshape(-1, 1)).ravel()
test_df["difference_from_predicted"] = abs(test_df["time_diff"] - test_df["predicted_time_diff"])
test_df["difference_from_predicted"].describe()

count     197.000000
mean      334.652296
std       358.158903
min         0.221039
25%        82.357941
50%       203.409363
75%       441.293396
max      1742.872314
Name: difference_from_predicted, dtype: float64